<a href="https://colab.research.google.com/github/KSY1526/SSUDA_LOTTEs/blob/main/Daniel/LPOINT_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# '''
# function ClickConnect(){
#     console.log("코랩 연결 끊김 방지"); 
#     document.querySelector("colab-toolbar-button#connect").click() 
# }
# setInterval(ClickConnect, 60 * 1000)
# '''

In [65]:
!pip install fastpivot
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi
!head /proc/cpuinfo
!head -n 3 /proc/meminfo

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
MemTotal:       26692024 kB
MemFree:        24361264 kB
MemAvailable:   25103316 kB


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm import tqdm
import warnings; warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import pyarrow as pa
# import pyarrow.csv as csv
from fastpivot import pivot_table

def class_mapping(df, col, verbose=True):
    class_map = {lbl:idx for idx, lbl in enumerate(sorted(df[col].unique()))}
    if verbose : print(class_map)
    return df[col].map(class_map)

path = "/content/drive/MyDrive/my_data/LPOINT/"

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
inputs = range(10)
def processInput(i):
    return i * i
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

In [ ]:
print(pd.__version__) # 1.3.5
print(pa.__version__) # 6.0.1

1.3.5
6.0.1


In [ ]:
pdde_cust = pd.read_csv(path+"/LPOINT_BIG_COMP_PDDE_CUST.csv", header=0, encoding="utf-8") # 고객정보까지 다 가져옴
# pdde_cust = pd.read_csv(path+"/LPOINT_BIG_COMP_PDDE.csv", header=0, encoding="utf-8") # 상품 구매정보만 가져옴

In [ ]:
def make_rct_parquet(i):    
    # chunk 단위로 고객 영수증 인덱스 추출
    if i != len(custs)//chunk_num:
        temp = pdde_cust.loc[pdde_cust["cust"].isin(custs[i*chunk_num : (i+1)*chunk_num])]
    else:
        temp = pdde_cust.loc[pdde_cust["cust"].isin(custs[(i+1)*chunk_num:])]

    pd_c_df = pivot_table(temp,
                          index="rct_no", columns="pd_c", values="buy_ct",
                          aggfunc='sum', fill_value=0).astype("uint16").reset_index() # 20sec ~ 40sec
    del temp
    pd_c_df = pd.concat((df, pd_c_df), axis=0)
    pd_c_df.values[pd_c_df.isna()] = 0 # fast fillna 30sec

    pd_c_df.to_parquet(path + f"LPOINT_BIG_COMP_PC_C_SUMS_{i}.parquet",
                       index=False,
                       engine="pyarrow", compression=None,
                       ) # 저장 시간: 20sec ~ 30sec

chunk_num = 1000
custs = pdde_cust.cust.unique()
df = pd.DataFrame(columns=["rct_no"]+[f"PD{str(i).zfill(4)}" for i in range(1,1934)])

_ = Parallel(n_jobs=num_cores)(delayed(make_rct_parquet)(i) for i in tqdm(range(len(custs)//chunk_num + 1)))

In [ ]:
additional_data = pdde_cust.loc[:, ["rct_no", "cust", "de_dt", "de_hr", "ma_fem_dv", "ages", "chnl_dv", "zon_hlv", "cop_c"]]\
                           .drop_duplicates("rct_no", keep="first")

additional_data["chnl_dv"] = class_mapping(additional_data, "chnl_dv")
additional_data["ma_fem_dv"] = class_mapping(additional_data, "ma_fem_dv")
additional_data["ages"] = class_mapping(additional_data, "ages")

additional_data["datetime"] = additional_data["de_dt"].apply(str).str[:4] \
                                + "-" + additional_data["de_dt"].apply(str).str[4:6] \
                                + "-" + additional_data["de_dt"].apply(str).str[6:8] \
                                + " " + additional_data["de_hr"].apply(lambda x: str(x).zfill(2))
additional_data["datetime"] = pd.to_datetime(additional_data["datetime"])
additional_data = additional_data.drop(["de_dt", "de_hr"], axis=1)

additional_data["month"] = additional_data.datetime.dt.month
additional_data["dayofweek"] = additional_data.datetime.dt.dayofweek
additional_data = pd.get_dummies(data=additional_data.loc[:, ["rct_no", "cust", "datetime", "ma_fem_dv", "chnl_dv", "ages", "zon_hlv", "cop_c", "month", "dayofweek"]],
                                 columns=["ages", "zon_hlv", "cop_c", "month", "dayofweek"])


additional_data = additional_data.set_index("rct_no")
additional_data.head(3)

{1: 0, 2: 1}
{'남성': 0, '여성': 1}
{'20대': 0, '30대': 1, '40대': 2, '50대': 3, '60대': 4, '70대': 5}


,cust,datetime,ma_fem_dv,chnl_dv,ages_0,ages_1,ages_2,ages_3,ages_4,ages_5,...,month_10,month_11,month_12,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
rct_no,,,,,,,,,,,,,,,,,,,,,
A01000001113,M430112881,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
A01000003148,M430112881,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
A01000004946,M430112881,2021-01-01 10:00:00,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
i = 0
df = pd.read_parquet(path+f"/LPOINT_BIG_COMP_PC_C_SUMS_{i}.parquet", engine="pyarrow")\
                     .set_index("rct_no")\
                     .join(additional_data)
len(df.cust.unique())
# 1000

1000

In [ ]:
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def make_rct_tensor(i, recent_rcts, cust_df):
    global features

    if len(cust_df) > i+1: # 고객의 남은 영수증이 1개 이하인 경우 None 반환
        temp = cust_df.iloc[i : i+recent_rcts].copy()
        temp["date_from"] = (temp.datetime.iloc[0] - temp.datetime).dt.days # 최근 구입일로부터 몇일만에 구입한 영수증인지. ex) [0]: 01/22, [1]: 02/11, [2]: 02/13 => [0]: 0, [1]: 20, [2]: 22
        temp = temp[features] # 피쳐 집합으로 원하는 피쳐를 골라 저장

        if 1 < temp.shape[0] < recent_rcts: # 고객의 남은 영수증이 2개 이상, 지정된 "최근 영수증 개수" 미만인 경우
            # "영수증 텐서" 생성, 부족한 영수증 데이터는 -1로 채움.
            temp = np.concatenate((temp.values,
                            -np.ones((recent_rcts - temp.shape[0], temp.shape[1]))),
                            axis=0,
                            dtype=np.int16,
                            casting="unsafe",
                            )
    
        else : # 고객의 받아온 영수증이 지정된 "최근 영수증 개수"인 경우
            # "영수증 텐서" 생성
            temp = temp.values.astype(np.int16)
        return  temp
    else:
        return None

In [ ]:
recent_rcts = 10 # 한 텐서 안에 들어갈 "최근 영수증 개수"

for j in range(len(custs)//chunk_num + 1):
    print("="*50+"\n"+"="*50)
    print(f"{j}번째 데이터셋을 생성중입니다.")

    # j번째 parquet 불러오기
    df = pd.read_parquet(path+f"/LPOINT_BIG_COMP_PC_C_SUMS_{j}.parquet", engine="pyarrow")\
                        .set_index("rct_no")\
                        .join(additional_data)

    # 피쳐 집합 생성
    features = [col for col in df.columns if col not in ["rct_no", "cust", "datetime"]] + ["date_interval", "date_from"]

    # 데이터셋 생성
    tensor_data = [] # stack할 후보군을 담을 리스트

    # 고객, 시간별 "영수증 텐서" 생성
    for cust_idx in tqdm(df.cust.unique()): # 데이터 프레임 내 모든 고객들에 대하여
        cust_df = df.loc[df.cust.isin([cust_idx])].sort_values(by="datetime", ascending=False).reset_index() # 고객별 시간순서대로 정렬
        cust_df["date_interval"] = (cust_df.datetime - cust_df.datetime.shift(-1)).dt.days.fillna(-1) # 몇일만에 새로운 구입을 하였는지

        results = Parallel(n_jobs=num_cores)(delayed(make_rct_tensor)(i, recent_rcts, cust_df) for i in range(len(cust_df)))
        tensor_data += [t for t in results if t is not None]
            
        # break
    tensor_data = np.stack(tensor_data, axis=0) # 한 csv내 "영수증 텐서"를 array 로 Stack
    print(f"{j}번째 영수증 텐서의 Shape : {tensor_data.shape}")
    np.save(path+f"/LPOINT_BIG_COMP_TORCH_np{j}.npy", tensor_data)
    # del tensor_data

# # data_len * recent_rcts * (1933 + K)

0번째 데이터셋을 생성중입니다.


100%|██████████| 1000/1000 [10:33<00:00,  1.58it/s]


0번째 영수증 텐서의 Shape : (31904, 10, 1985)
1번째 데이터셋을 생성중입니다.


100%|██████████| 1000/1000 [10:06<00:00,  1.65it/s]


1번째 영수증 텐서의 Shape : (29861, 10, 1985)
2번째 데이터셋을 생성중입니다.


...

100%|██████████| 1000/1000 [05:49<00:00,  2.86it/s]


12번째 영수증 텐서의 Shape : (14048, 10, 1985)
13번째 데이터셋을 생성중입니다.


100%|██████████| 1000/1000 [05:33<00:00,  3.00it/s]


13번째 영수증 텐서의 Shape : (13559, 10, 1985)
14번째 데이터셋을 생성중입니다.


 67%|██████▋   | 674/1000 [03:44<01:36,  3.37it/s]

In [ ]:
# 위의 루프에서 하면 너무 오래걸려서 추가로 시행한 부분

np_tensor_data = [] # np자료끼리 묶을 리스트
for j in range(len(custs)//chunk_num + 1):
    tensor_data = np.load(path+f"/LPOINT_BIG_COMP_TORCH_np{j}.npy")
    np_tensor_data += tensor_data
    print(f"{j}번째 영수증 텐서의 Shape : {tensor_data.shape}")
np_tensor_data = np.stack(np_tensor_data, axis=0)
np.save(path+f"/LPOINT_BIG_COMP_TORCH_np_Total.npy", np_tensor_data)
# np_tensor_data = np.load(path+f"/LPOINT_BIG_COMP_TORCH_np{j}.npy")

# 이후 수정 예정

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, id, train_mode = True, transforms = None, n = None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.df = df
        self.id = id
        self.n = n

    def __len__(self):
        return len(self.id)

    def __getitem__(self, idx):
        id = self.id[idx]
        input = self.df.loc[list(map(lambda tmp : (id in tmp), tmp_list)),:]

        if self.train_mode:
            output = input.iloc[:,:-4]
            return torch.tensor(np.array(input)), torch.tensor(np.array(output))

        else:
            return torch.tensor(np.array(input))